# Setup

## Imports

In [2]:
import QuantumToolbox as qt
import SuperconductingCavities as SC
import CairoMakie as cm
using Revise
using Dates
import JSON3

using ProgressMeter

import Optim as opt

using Logging
using MiniLoggers

┌ Info: new driver key :netcdf, updating backendlist.
└ @ NetCDFExt /home/gr400/.julia/packages/YAXArrayBase/Xj43k/ext/NetCDFExt.jl:81


## Logger Setups

In [3]:
# MiniLogger(minlevel = MiniLoggers.Info) |> global_logger
InfoLogger = MiniLogger(minlevel = MiniLoggers.Info)
ProgressLogger = MiniLogger(minlevel = LogLevel(-1))
DebugLogger = MiniLogger(minlevel = MiniLoggers.Debug)

global_logger(ProgressLogger)

Base.CoreLogging.SimpleLogger(VSCodeServer.IJuliaCore.IJuliaStdio{Base.PipeEndpoint, typeof(VSCodeServer.io_send_callback)}(IOContext(Base.PipeEndpoint(RawFD(21) open, 0 bytes waiting)), VSCodeServer.io_send_callback), Info, Dict{Any, Int64}())

# Fitting g

In [4]:
device_config = JSON3.read("device_config_files/config_multimode.json", Dict{Any, Any});

In [5]:
number_of_modes = length(device_config["device"]["soc"]["storage"]["freqs"])

E_J = 26.96976142643705
E_C = 0.10283303447280807

E_oscs = device_config["device"]["soc"]["storage"]["freqs"]./1000

χ_e(model) = (model.dressed_energies[(1,1)]-model.dressed_energies[(0,1)])-(model.dressed_energies[(1,0)]-model.dressed_energies[(0,0)]);
χ_f(model) = (model.dressed_energies[(2,1)]-model.dressed_energies[(0,1)])-(model.dressed_energies[(2,0)]-model.dressed_energies[(0,0)]);

χ_e_targets = device_config["device"]["soc"]["storage"]["chi_e"]./1000
χ_f_targets = device_config["device"]["soc"]["storage"]["chi_f"]./1000
initial_g = 0.02#0.026184375000000013;

0.02

In [6]:
function function_to_minimize_maker(E_osc, χ_e_target, χ_f_target)
    function to_min(g)
        model = SC.Circuits.Transmon_Resonators.init(E_C, E_J, E_osc, g, 10, 10)
        return (χ_e_target - χ_e(model))^2+(χ_f_target - χ_f(model))^2
    end
    return to_min
end

function_to_minimize_maker (generic function with 1 method)

In [7]:
model_list = []

@showprogress for i in 1:number_of_modes
    to_min = function_to_minimize_maker(E_oscs[i], χ_e_targets[i], χ_f_targets[i])
    result = opt.optimize(to_min, [initial_g], opt.LBFGS())

    Model_Name = "Mode$i"
    Cavity_Names = ["Mode$i"]
    model = SC.Circuits.Transmon_Resonators.init(E_C, E_J, E_oscs[i], result.minimizer[1], 10, 10; Model_Name = Model_Name, Cavity_Names = Cavity_Names, Save_Path = "ModelSaves/$Model_Name/")
    model.Stuff["op_drive_params"] = Dict{Any, Any}() 
    push!(model_list, model)
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:11


In [8]:
for i in 1:length(model_list)
    SC.Utils.save_model(model_list[i])
end